<a href="https://colab.research.google.com/github/Asymmetric-OG/amoghpanthangi_spider_task_1/blob/main/SPML_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.modules as nn
import numpy
import pandas
import matplotlib.pyplot as plt
import torchmetrics
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchmetrics.classification import Accuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #to access gpu if available

#Introducing standard transformations for Resnet18 as given in its documentation, this when implemented converts images into tensors and normalizes them to create expected input

trans = transforms.Compose([transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR), transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                 std=[0.229, 0.224, 0.225])])

#creation of datasets
full_dset = datasets.ImageFolder(root="/content/Vehicles/Vehicles", transform=trans) #ImageFolder assigns labels to each class of vehicles and saves the dataset in form of tuples after applying transforms to them (image tensor, class label)

#splitting the complete dataset into training dataset and testing dataset using random_split from torch.utils.data
train_size = int(0.9 * len(full_dset))
test_size = len(full_dset) - train_size
train_dset, test_dset = random_split(full_dset, [train_size, test_size])

#creating dataloaders where batches of 32 are made to make training efficient, this is standard for datasets of this size.
#Shuffle is turned on to improve generalization and removes possible bias due to a certain order. Makes the training more effective
train_dloader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_dloader = DataLoader(test_dset, batch_size=32, shuffle=True)


model = models.resnet18(pretrained=True) #imports pretrained resnet18 model into the model variable
model.fc = nn.Linear(in_features=512 , out_features=7) #changes final output layer of resnet18 to accomodate 7 classes.
model = model.to(device) #pushing the model to GPU

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
'''optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)'''

epochs= 8
train_losses = []
epoch_count = []

for epoch in range(epochs) :

  model.train() #training mode
  train_loss = 0.0
  for input, out_true in train_dloader:
    input, out_true = input.to(device), out_true.to(device) #Pushing image and its label to GPU
    out_pred = model(input) #forward pass, output is an array of 7 different scores (logits), 1 for each class
    batch_avg_loss = loss_fn(out_pred,out_true) #computes the average loss of a batch based on loss function of choice
    optimizer.zero_grad() #cleares gradient cache so that it doesnt get accumulated
    batch_avg_loss.backward() #backward pass to compute gradient of loss, meaning how much the weights should be changed
    optimizer.step() #implements the change in weights of the model
    train_loss+=batch_avg_loss.item() #computes total loss of an epoch. .item() converts tensor to value
  train_losses.append(train_loss)
  epoch_count.append(epoch)
  print(f"Epoch {epoch+1}: Train Loss = {train_loss:.2f}")

#Plotting the training losses as epochs progress

plt.plot(epoch_count, train_losses, label='Training Loss') #specifies the x and y variable
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.legend()
plt.show()

accuracy = Accuracy(task="multiclass", num_classes=7)  #sets up accuracy from torchmetrics for a 7 class multiclass problem
accuracy = accuracy.to(device) #pushing accuracy to GPU

model.eval() #evaluation(testing) mode
accuracy.reset() #avoiding accumulation
with torch.inference_mode() : #disables functionality like gradient calculation not required during testing process to make it more efficient

  for test_input, test_out in test_dloader:  #images and their labels
    test_input, test_out = test_input.to(device), test_out.to(device) #pushing images and their labels to gpu
    test_pred = model(test_input)
    prediction = test_pred.argmax(dim=1) #finding the highest score and class/label corresponding to it, this is the prediction made by the model
    accuracy.update(prediction, test_out)
  acc = accuracy.compute().item() #accuracy is then computed wrt to the correct label of the image
  print(f"Test Accuracy: {acc * 100}%")






ModuleNotFoundError: No module named 'torchmetrics'

In [ ]:
import zipfile
import os

with zipfile.ZipFile("Vehicles.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Vehicles")

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall